In [ ]:
import cv2
import numpy as np

# Step 1: Read and process the video
video_path = 'C:/Users/HP/Desktop/image assign/Project Video.mp4'
cap = cv2.VideoCapture(video_path)

# Step 2: Preprocessing & noise removal
def preprocess_frame(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply GaussianBlur for noise removal
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred


# Step 3: Segmentation
def segment_frame(frame):
    # Apply adaptive thresholding to detect subtitles
    _, thresh = cv2.threshold(frame, 200, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Extract words from contours
    words = []
    for contour in contours:
        # Calculate contour area
        area = cv2.contourArea(contour)
        
        # Filter out shapes based on contour area
        if area > 100:
            # Get the minimum bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Filter out shapes based on aspect ratio
            aspect_ratio = w / h
            if 0.2 < aspect_ratio < 5:
                words.append(((x, y), (x + w, y + h)))
    
    return thresh, words

# Step 4: Calculate bounding boxes dimensions
def calculate_bounding_boxes(words):
    # Sort words by y-coordinate to handle multiline subtitles
    sorted_words = sorted(words, key=lambda x: x[0][1])

    # Group words into lines based on y-coordinate proximity
    lines = []
    current_line = [sorted_words[0]]
    for word in sorted_words[1:]:
        if abs(word[0][1] - current_line[-1][1][1]) < 20:  # Adjust this threshold as needed
            current_line.append(word)
        else:
            lines.append(current_line)
            current_line = [word]
    lines.append(current_line)

    # Calculate bounding boxes for lines and words
    subtitle_boxes = [cv2.boundingRect(np.array([point for box in line for point in box])) for line in lines]
    word_boxes = [cv2.boundingRect(np.array(word)) for line in lines for word in line]

    return subtitle_boxes, word_boxes

# Step 5: Draw bounding boxes on the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error reading video or end of video reached.")
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_frame(frame)

    # Segment the frame to detect subtitles and words
    thresh, words = segment_frame(preprocessed_frame)

    # Calculate bounding box dimensions
    subtitle_boxes, word_boxes = calculate_bounding_boxes(words)

    # Draw bounding boxes on the frame
    for box in subtitle_boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red for subtitles

    for box in word_boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green for words

    # Display the frame with bounding boxes
    cv2.imshow('Video with Bounding Boxes', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()